¿Qué es la regresión lineal múltiple?
==========================

¿Qué pasa si a la regresión lineal simple le agregamos variables? a esto se le conoce como regresión lineal múltiple, solo que ahora el modelo busca relacionar dos o más variables independientes con una dependiente. Se describe como:

<center>

$\hat{Y} = \beta_0 + \beta_1 X_1 + ... + \beta_n X_n$

</center>

Además de las condiciones requeridas por los modelos lineales simples, los modelos de correlación lineal múltiple necesitan cumplir con:
* No colinialidad: las variables predictoras no deben tener colinialidad entre ellas.
* Parsimonia: el mejor modelo es aquel capaz de explicar con mayor precisión la variabilidad observada con el menor número de predictores.

Selección de predictores
==========================
Los métodos más comunes para seleccionar variables son:
* *Eliminación hacia atrás (Backward Stepwise Regression)*: se introducen todas las variables en la ecuación y se excluyen iterativamente las que tengan menor valor p.
* *Selección hacia adelante (Fordward Stepwise Regression)*: se introducen secuencialmente las variables en el modelo que tengan la mayor correlación con la variable dependiente.
* Método paso a paso (Stepwise Regression): se combinan los procedimientos anteriores para introducir variables con valor p más pequeño y estas pueden ser eliminadas según el valor R² obtenido. 

Nota: el **valor p** es una probabilidad usada cuando se trabaja con pruebas de hipótesis. Como resumen, entre menor sea el valor p asociado a una variable, esta describe mejor a la variable dependiente.

Implementación
==========================

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Importamos el dataset
dataset = pd.read_csv('./datasets/50_Startups.csv')
X = dataset.iloc[:, :-1].values
Y = dataset.iloc[:, 4].values

In [2]:
# Trabajamos las variables categóricas
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Codificamos las variables categóricas ordinales
labelencoder_X = LabelEncoder()
X[:, 3] = labelencoder_X.fit_transform(X[:, 3])

# Codificamos las variables categóricas nominales
ct = ColumnTransformer([("Country", OneHotEncoder(), [3])], remainder = 'passthrough')
X = ct.fit_transform(X)
X = X[:, 1:]

# Convertimos los datos a float para evitar problemas de compatibilidad de datos
X = X.astype(float)
Y = Y.astype(float)

In [3]:
# Dividimos el dataset en conjunto de entrenamiento y conjunto de prueba
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [4]:
# Entrenamos nuestro modelo
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, Y_train)

# Y hacemos las predicciones
y_pred = regressor.predict(X_test)

In [11]:
# Analicemos los resultados obtenidos
from sklearn.metrics import r2_score
r2 = r2_score(Y_test, y_pred)
r2_adj = 1 - (1 - r2) * (50 - 1) / (50 - 5 - 1)
print('R2: ', r2)
print('R2 ajustado: ', r2_adj)

R2:  0.9347068473282921
R2 ajustado:  0.9272871708883254


In [13]:
import statsmodels.api as sm

X = np.append(arr = np.ones((50, 1)).astype(int), values = X, axis = 1)
X_opt = X[:, [0, 1, 2, 3, 4, 5]]
SL = 0.05

regressor_OLS = sm.OLS(endog = Y, exog = X_opt).fit() # Método de los mínimos cuadrados ordinarios
print(regressor_OLS.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Wed, 01 Feb 2023   Prob (F-statistic):           1.34e-27
Time:                        19:58:38   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       5.013e+04   6884.820      7.281      0.0

In [24]:
SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]

# Backward Elimination con solo el valor p

def backwardElimination(x, y, sl):    
    
    numVars = len(x[0])    

    for i in range(0, numVars):        
    
        regressor_OLS = sm.OLS(y, x.tolist()).fit()        
        maxVar = max(regressor_OLS.pvalues).astype(float)        
    
        if maxVar > sl:            
            for j in range(0, numVars - i):                
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):                    
                    x = np.delete(x, j, 1)    

        print(f"\n\nCon {len(X_opt[0])} variables, el máximo valor p es: {maxVar}")
        print(regressor_OLS.summary())
    
    return x 

model = sm.OLS(Y, backwardElimination(X_opt, Y, SL)).fit()



Con 6 variables, el máximo valor p es: 0.9897941241608603
                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.945
Method:                 Least Squares   F-statistic:                     169.9
Date:                Wed, 01 Feb 2023   Prob (F-statistic):           1.34e-27
Time:                        20:16:15   Log-Likelihood:                -525.38
No. Observations:                  50   AIC:                             1063.
Df Residuals:                      44   BIC:                             1074.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------

In [25]:
# En la última iteración de la eliminación hacia atrás, el valor p 
# más alto que eliminamos era muy cercano respecto al nivel de significancia
# y causa una reducción en el valor R2 ajustado, por lo que no es necesario eliminarla

# Backward Elimination considerando al valor p y a R2 ajustado

SL = 0.05
X_opt = X[:, [0, 1, 2, 3, 4, 5]]

numVars = len(X_opt[0])    
temp = np.zeros((50,6)).astype(int)   

def backwardElimination(x, y, SL):    

    numVars = len(x[0])    
    temp = np.zeros((50,6)).astype(int)    
    
    for i in range(0, numVars):     

        regressor_OLS = sm.OLS(y, x.tolist()).fit()        
        maxVar = max(regressor_OLS.pvalues).astype(float)        
        adjR_before = regressor_OLS.rsquared_adj.astype(float)        
        
        print(f"\n\nCon {len(X_opt[0])} variables, el máximo valor p es: {maxVar}")
        print(regressor_OLS.summary())

        if maxVar > SL:            
            for j in range(0, numVars - i):                
                if (regressor_OLS.pvalues[j].astype(float) == maxVar):    

                    temp[:,j] = x[:, j]                    
                    x = np.delete(x, j, 1)                    
                    tmp_regressor = sm.OLS(y, x.tolist()).fit()                    
                    adjR_after = tmp_regressor.rsquared_adj.astype(float)                    
                    
                    if (adjR_before >= adjR_after):                        
                        x_rollback = np.hstack((x, temp[:, [0, j]]))                        
                        x_rollback = np.delete(x_rollback, j, 1)                        
                        return x_rollback                    

    return x 

model = sm.OLS(Y, backwardElimination(X_opt, Y, SL)).fit()

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.950
Model:                            OLS   Adj. R-squared:                  0.948
Method:                 Least Squares   F-statistic:                     450.8
Date:                Wed, 01 Feb 2023   Prob (F-statistic):           2.16e-31
Time:                        20:18:04   Log-Likelihood:                -525.54
No. Observations:                  50   AIC:                             1057.
Df Residuals:                      47   BIC:                             1063.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4.698e+04   2689.933     17.464      0.0